<a href="https://colab.research.google.com/github/cielo229/AI/blob/main/Fine_tuning_a_model_with_the_Trainer_Wandb_Chinese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Fine-tuning a model with the Trainer Wandb Chinese
##安裝套件

In [3]:
!pip install datasets evaluate transformers[sentencepiece] --quiet
!pip install accelerate -U --quiet
!pip install wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.1/281.1 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("sepidmnorozy/Chinese_sentiment")
checkpoint = "google-bert/bert-base-chinese"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Generating train split:   0%|          | 0/12348 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2591 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4896 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/12348 [00:00<?, ? examples/s]

Map:   0%|          | 0/2591 [00:00<?, ? examples/s]

Map:   0%|          | 0/4896 [00:00<?, ? examples/s]

##定義Trainer的參數
我們定義我們之前的第一步Trainer是定義一個類別，其中包含將用於訓練和評估的TrainingArguments所有超參數。為求簡單，這邊Trainer提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [6]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer", num_train_epochs=3, report_to="wandb")

使用AutoModelForSequenceClassification有兩個標籤的類別

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

##定義Trainer
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、training_args、訓練和驗證資料集、data_collator和分詞器

In [8]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

##說明
這樣傳遞分詞器時，訓練器使用的預設data_collator將是之前定義的DataCollatorWithPadding，因此可以跳過data_collator=data_collator這一行

##開始訓練
呼叫Trainer中的train()方法

In [9]:
trainer.train()

wandb: Currently logged in as: jessylou1028 (cielo229). Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.469900
1000,0.382300
1500,0.396700
2000,0.365200
2500,0.312500
3000,0.322700
3500,0.253000
4000,0.245900
4500,0.209500


TrainOutput(global_step=4632, training_loss=0.32613343443071696, metrics={'train_runtime': 1450.5236, 'train_samples_per_second': 25.538, 'train_steps_per_second': 3.193, 'total_flos': 3856159738466400.0, 'train_loss': 0.32613343443071696, 'epoch': 3.0})

In [10]:
trainer.save_model("./testmodel")

In [11]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="./testmodel")

In [12]:
classifier(
    [
        "真的很棒",
        "只會來這一次了",
    ]
)

[{'label': 'LABEL_1', 'score': 0.9988129138946533},
 {'label': 'LABEL_0', 'score': 0.9532483220100403}]

In [13]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}